In [106]:
import numpy as np
from numba import njit
from scipy.ndimage import laplace
from tqdm import tqdm

BASESHIFT = np.array([-1, 0, 1])
NEIGHBORS = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])

In [78]:
class State:
    def __init__(self, L, N, q):
        self.L = L # lattice size
        self.N = N # number of anyons
        self.q = q # anyons
        self.Φ = np.zeros((L, L), dtype = np.float32) # field

In [79]:
def init_state(L, p_anyon):
    vert_anyons = (np.random.rand(L, L) < p_anyon).astype(int)
    vert_anyons += np.roll(vert_anyons, 1, axis=0)

    horiz_anyons = (np.random.rand(L, L) < p_anyon).astype(int)
    horiz_anyons += np.roll(horiz_anyons, 1, axis=1)
    
    q = (vert_anyons + horiz_anyons) % 2
    N = np.sum(q)
    return State(L, N, q)

In [88]:
def update_field(state, η):
    state.Φ += η / 4 * laplace(state.Φ, mode = 'wrap')
    state.Φ += state.q

In [ ]:
def update_anyon(state):
    for x, y in np.transpose(np.nonzero(state.q)):
        idx = x * state.L + y
        shifts = np.add.outer(BASESHIFT * state.L, BASESHIFT).flatten()
        neighborhood = state.phi.take(shifts + idx, mode = 'wrap')
        neighborhood *= NEIGHBORS
        dir = neighborhood.argmax()

In [81]:
mystate = init_state(10, 0.5)

In [90]:
testarr = np.arange(25).reshape(5, 5) + 1
testarr

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [110]:
testarr.argmax(keepdims = True)

array([[24]], dtype=int64)